In [2]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from gensim import corpora
from collections import defaultdict
from gensim import models
from gensim import similarities

from google.colab import drive
drive.mount('/content/drive')

dataset = open('/content/drive/MyDrive/Colab Notebooks/data.txt', 'r')
documents = []
for line in dataset:
  documents.append(line)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
stoplist = set('for a of the and to in'.split())
texts = [
    [word for word in document.lower().split() if word not in stoplist]
    for document in documents
]

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)], [], [(3, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1)], [], [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 2), (11, 1), (15, 1), (18, 1), (19, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [], [(1, 1), (2, 1), (6, 1), (8, 1), (12, 1), (29, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1)], [], [(6, 1), (9, 4), (10, 1), (14, 1), (17, 1), (18, 4), (19, 1), (20, 1), (23, 1), (24, 1), (25, 1), (29, 3), (30, 1), (31, 1), (32, 2), (35, 1), (36, 1), (39, 1), (40, 1), (52, 1), (53, 1), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1)], [(0, 1), (11, 1), (16, 1), (18, 2), (22, 1), (31, 1), (50, 1), (56, 1), (57, 1)

In [7]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

doc = "climate change"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
print(vec_lsi)

[(0, 0.14371374026928516), (1, 0.17811621243523354)]


In [12]:
index = similarities.MatrixSimilarity(lsi[corpus], num_features = len(dictionary))
sims = index[vec_lsi]

sims = sorted(enumerate(sims), key= lambda item: -item[1])

print('{:8} {:} {:}'.format('Score', '\t', 'Document'))
for doc_position, doc_score in sims:
    print('{:.6f} {:} {:}'.format(doc_score, '\t', documents[doc_position]))

Score    	 Document
0.999869 	 The firm’s latest climate initiative builds on its 2020 pledge to achieve net-zero financed emissions by 2050, in line with the Paris Agreement. Last year, Morgan Stanley also joined the Steering Committee of the Partnership for Carbon Accounting Financials to develop the tools and methodologies to measure and disclose our carbon-related activities. Both efforts were the first of their kind for a large U.S. bank.

0.998071 	 Morgan Stanley’s Climate Change Commitments

0.991820 	 Morgan Stanley’s other climate commitments include achieving carbon neutrality across global operations by 2022—a goal that was set in 2017—to power 100% of global operational electricity needs from renewable sources and offset any remaining emissions. To achieve this, the firm is exploring on-site power generation, securing power-purchase agreements and purchasing renewable energy credits and carbon offsets.

0.968819 	 Morgan Stanley also joined the United Nations-convened Net-